In [1]:
!pip install -U langchain-mcp-adapters langgraph langchain-openai

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langchain.agents import create_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

server_params = StdioServerParameters(
    command="python3",
    args=["./math_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "(3 + 5) x 12는 얼마지?"})

agent_response

{'messages': [HumanMessage(content='(3 + 5) x 12는 얼마지?', additional_kwargs={}, response_metadata={}, id='726d3279-8cda-4dea-8b11-3e803b4a0a19'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 78, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3683ee3deb', 'id': 'chatcmpl-D39LLS1JWmOI1h7ZMg6kTWH1LLzam', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c06f5-5dc0-7592-badc-0b58abb2d48d-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_vVHXKIzzYvB03EUav2bxOEzm', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 12, 'b': 1}, 'id': 'call_JZkXVi14MvfJKIUaj9owJMZe',

In [5]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

client = MultiServerMCPClient(
    {
        "math": {
            "command": "python3",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["./math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    }
) 
agent = create_agent(model, await client.get_tools())
math_response = await agent.ainvoke({"messages": "(3 + 5) x 12는 얼마지?"})
weather_response = await agent.ainvoke({"messages": "뉴욕의 날씨는 어때?"})

In [6]:
math_response

{'messages': [HumanMessage(content='(3 + 5) x 12는 얼마지?', additional_kwargs={}, response_metadata={}, id='5c01a663-3071-464f-9cd3-79e911710654'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 97, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D39MGoPof375MVH5ibx5pA741txFC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c06f6-401d-7c23-bc20-ba1d777f225b-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_XFwlvyu7dvUhKM5wsLDVlEzH', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 97, 'output_tokens': 17, 'total_tok

In [7]:
weather_response

{'messages': [HumanMessage(content='뉴욕의 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='13559de2-ac33-4004-8db7-b060f3f6636d'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 93, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D39MKPSeI4yNfvPZ2SN1dkKuyDR78', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c06f6-4e1f-7603-be9c-698f12be71b0-0', tool_calls=[{'name': 'get_weather', 'args': {'location': '뉴욕'}, 'id': 'call_MOMqinhDkleLZ3G02zHp5XjL', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 93, 'output_tokens': 15, 'total_